<a href="https://colab.research.google.com/github/JayJay95/CodePractice/blob/master/VulnerabilityClassifier_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My\ Drive/After4thYear/Belfast/MSc\ Cybersec/Research\ Project/Colab Notebooks

/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks


In [0]:
import os
import re
import fnmatch
import argparse
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import test_cases
from copy import deepcopy
import pdb

In [0]:
clean_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Clean_Opseq'
vuln_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Vuln_Opseq'

In [0]:
parser = argparse.ArgumentParser(description='Vulnerability Classifier')
# parser.add_argument('--max_opcode_seq_len', action='store', type=int, help='use different versions of network', default=8192)
# parser.add_argument('--min_opcode_seq_len', action='store', type=int, help='use different versions of network', default=32)
parser.add_argument('--lr', action='store', type=float, help='use different versions of network', default=1e-3)
parser.add_argument('--epochs', action='store', type=int, help='use different versions of network', default=50)
opt = parser.parse_args('')
print(opt)

Namespace(epochs=50, lr=0.001)


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
def find_files(files, dirs=[], extensions=[]): # recursively find files in directories
    new_dirs = []
    for d in dirs:
        try:
            new_dirs += [ os.path.join(d, f) for f in os.listdir(d)] # check in all directories except testcasesupport 
        except OSError:
            if os.path.splitext(d)[1] in extensions:
                files.append(d)

    if new_dirs:
        find_files(files, new_dirs, extensions)
    else:
        return  

def read_file(filename):
    opcode_count = 0
    line_list = []
    with open(filename, mode='rt', encoding='utf8') as f:
        content = f.readlines()        
    for line in content:
        opcode_seq = []     
        for c in range(0, len(line) - 1, 2):
            #print(line[c:(c+2)],int(line[c:(c+2)], 16))
            opcode_seq.append(int(line[c:(c+2)], 16) + 1) # add one here so that the zero'th embedding is reserved for 'blank' i.e. no instruction whatsoever not even no-op			
            opcode_count += 1
            # to save training time we only read 
            # the first opt.max_opcode_seq_len opcodes of each file
            
            # if opcode_count >= opt.max_opcode_seq_len:
            #     return opcode_seq
        line_list.append(opcode_seq)
    return line_list

def read_dataset():
    vuln = []
    clean = []
    clean_opseq_files = []
    vuln_opseq_files = []
    # min_file_len = opt.min_opcode_seq_len #ignore opcode seq files shorter than this
    find_files(clean_opseq_files, dirs=[clean_opseq_folder], extensions=['.clean'])
    for clean_file_pathname in clean_opseq_files:
        tmp = read_file(clean_file_pathname)
        # if len(tmp) >= min_file_len:
        clean.append(tmp)
    
    find_files(vuln_opseq_files, dirs=[vuln_opseq_folder], extensions=['.vuln'])
    for vuln_file_pathname in vuln_opseq_files:
        tmp = read_file(vuln_file_pathname)
        # if len(tmp) >= min_file_len:
        vuln.append(tmp)
    
    # flatten vuln and clean lists
    new_vuln = []
    for x in vuln:
        for y in x:
            new_vuln.append(y)
    
    new_clean = []
    for x in clean:
        for y in x:
            new_clean.append(y)
    return new_vuln, new_clean

In [0]:
def split_dataset(vuln, clean):
    #split the dataset into train, val, test sets
    #return the concatenated dataset and
    #indicies pointing to the train,val,test samples

    vuln_label = 0
    clean_label = 1
    dataset = deepcopy(clean) + deepcopy(vuln) # concatenate original clean and original vuln samples
    trial_dataset = deepcopy(clean) + deepcopy(vuln) 
    
    final_vuln_list = []
    for vuln_list in vuln:
        inner_vuln_list = []
        inner_vuln_list.append(vuln_list)
        inner_vuln_list.append(vuln_label)
        final_vuln_list.append(inner_vuln_list)
             
    final_clean_list = []
    for clean_list in clean:
        inner_clean_list = []
        inner_clean_list.append(clean_list)
        inner_clean_list.append(clean_label)
        final_clean_list.append(inner_clean_list)
    
    # split vuln samples randomly
    first_vuln_split = int(0.8 * len(final_vuln_list))
    second_vuln_split = int(0.1 * len(final_vuln_list))
    third_vuln_split = int(len(final_vuln_list) - (first_vuln_split + second_vuln_split))
    vuln_training_dataset, vuln_validation_dataset, vuln_testing_dataset = torch.utils.data.random_split(final_vuln_list, [first_vuln_split, second_vuln_split, third_vuln_split])

    # split clean samples randomly
    first_clean_split = int(0.8 * len(final_clean_list))
    second_clean_split = int(0.1*len(final_clean_list))
    third_clean_split = int(len(final_clean_list) - (first_clean_split + second_clean_split))
    clean_training_dataset, clean_validation_dataset, clean_testing_dataset = torch.utils.data.random_split(final_clean_list, [first_clean_split, second_clean_split, third_clean_split])

    # merge both vuln & clean training sets
    training_set = clean_training_dataset + vuln_training_dataset

    # get indices and labels from training set
    train_inds=[]
    train_labels=[]
    training_counter = 0
    
    for list in training_set:
        train_inds.append(training_counter)
        training_counter+=1
        train_labels.append(list[1])
        
    # merge both vuln & clean validation sets
    validation_set = clean_validation_dataset + vuln_validation_dataset

    # get indices and labels from validation set
    val_inds = []
    val_labels = []
    training_set_length = len(training_set)
    for list in validation_set:
        val_inds.append(training_set_length)
        training_set_length += 1
        val_labels.append(list[1])

    # merge both vuln & clean testing sets
    testing_set = clean_testing_dataset + vuln_testing_dataset

    # get indices and labels from testing set
    test_inds = []
    test_labels = []
    validation_set_length = len(training_set) + len(validation_set)
    for list in testing_set:
        test_inds.append(validation_set_length)
        validation_set_length += 1
        test_labels.append(list[1])
    return trial_dataset, (train_inds, train_labels), (val_inds, val_labels), (test_inds, test_labels)

In [0]:
class VulnerabilityDetectorNetwork(nn.Module):
    def __init__(self):
        super(VulnerabilityDetectorNetwork, self).__init__()

        self.num_of_embeddings = 205
        self.embedding_dimension = 8
        self.channels_in = 1
        self.channels_out = 64
        self.kernel_height_dimension = 1
        self.kernel_width_dimension = 8
        self.padding_height_dimension = 0
        self.padding_width_dimension = 0
        self.features_out = 2

        # self.emb1 = nn.Embedding(self.num_of_embeddings, self.embedding_dimension)
        # self.conv1 = nn.Conv2d(self.channels_in, self.channels_out, kernel_size=(self.kernel_height_dimension, self.kernel_width_dimension),
        #     padding=(self.padding_height_dimension, self.padding_width_dimension))
        # self.bn1 = nn.BatchNorm2d(64)
        # # self.drop_out = nn.Dropout()
        # self.lin1 = nn.Linear(self.channels_out, self.features_out)

        self.emb1 = nn.Embedding(205, 8)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1, 8), padding=(0, 0))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1, 1), padding=(0, 0))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(1, 1), padding=(0, 0))
        self.pool = nn.MaxPool2d(2, 2)
        self.drop_out = nn.Dropout(0.25)
        self.lin1 = nn.Linear(8000, 128)
        self.lin2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.emb1(x)
        # print(x.shape)
        x = x.unsqueeze(0)
        # print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.shape)
        x = self.pool(F.relu(self.conv3(x)))
        # print(x.shape)
        x = x.squeeze(3)
        # print(x.shape)
        x = x.view(x.shape[0], -1)
        # print(x.shape)
        x = self.drop_out(x)
        # print(x.shape)
        x = F.relu(self.lin1(x))
        # print(x.shape)
        x = self.drop_out(x)
        # print(x.shape)
        x = self.lin2(x)
        # print(x.shape)
        return x

In [0]:
def evaluate_network_performance(predictions, ground_truth):
    # given an array with the predicted values and the correct values
    # calculate the precision, recall and f-score
    cm = get_confusion_matrix(predictions, ground_truth)
    TP = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TN = cm[1][1]

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    classification_accuracy = (TP+TN) / (TP + TN + FP + FN)
    f_score = 2 * ((precision*recall)/(precision + recall))

    return precision, recall, f_score, classification_accuracy

In [0]:
def get_confusion_matrix(preds, truth):
    K = len(np.unique(truth)) # Number of classes 
    result = np.zeros((K, K))
    for i in range(len(truth)):
        result[preds[i]][truth[i]] += 1
    confusion_matrix = result
    return confusion_matrix

In [0]:
def test_network(net, dataset, tvt_set):
    # given a dataset and network compute the accuracy over the data_set

    inds = tvt_set[0]
    labels = tvt_set[1]

    # test = torch.utils.data.TensorDataset(inds, labels)
    # test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)
    
    criterion = nn.CrossEntropyLoss()

    test_loss = 0
    predictions = []

    with torch.no_grad():
        for i in range(len(inds)):
        # for i, (imgs, lbls) in enumerate(test_loader):
        #     imgs = Variable(imgs.view(-1, 1, 100))
            net.eval()

            example = torch.LongTensor(dataset[inds[i]]).unsqueeze(0)
            target = torch.LongTensor([labels[i]])
            
            example, target = example.to(device), target.to(device)

            output = net(example)
            # output = net(imgs)
            val,idx = torch.max(output,1)
            predictions.append(idx.item())

            test_loss = criterion(output, target).item()
            # test_loss = criterion(output, lbls).item()

        precision, recall, f_score, classification_accuracy = evaluate_network_performance(predictions, labels)

    return precision, recall, f_score, classification_accuracy

In [0]:
def train_network(net, dataset, train_set, val_set):
	
    train_inds = train_set[0]
    train_labels = train_set[1]
    # pdb.set_trace()

    # trainIndsTensor = torch.LongTensor(train_inds)
    # trainLblsTensor = torch.LongTensor(train_labels)
    # train = torch.utils.data.TensorDataset(trainIndsTensor, trainLblsTensor)
    # train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
    
    optimizer = optim.Adam(net.parameters(), lr=opt.lr)
    criterion = nn.CrossEntropyLoss()

    num_training_epochs = opt.epochs
    # n_iters = 10000
    # num_epochs = n_iters/(len(train_set)/batch_size)
    # num_epochs = int(num_epochs)
    for list in dataset:
        if len(list) < 1000:
            list.extend([0] * (1000- len(list)))


    for e in range(num_training_epochs):
        randperm = torch.randperm(len(train_inds))
        running_loss = 0
        for i in range(len(train_inds)):
        # for i, (images, labels) in enumerate(train_loader):
            # randperm = torch.randperm(len(images))
            # train = Variable(images.view(1, 100)).squeeze(0)
            # labels = Variable(labels)
            net.train()
            optimizer.zero_grad()

            example = torch.LongTensor(dataset[train_inds[randperm[i]]]).unsqueeze(0)
            target = torch.LongTensor([train_labels[randperm[i]]])
            # pdb.set_trace()
            # print ("Example: ", example.shape)
            example, target = example.to(device), target.to(device)

            net_output = net(example)
            # print("Net: ", net_output.shape)
            # net_output = net(train)
            # loss = criterion(net_output, target)
            # print(net_output)
            # target = target.view(1,-1)
            loss = criterion(net_output, target)
            # loss = criterion(net_output, torch.max(target, 1)[1])
            # max = torch.max(net_output)
            # print ("MAX: ", max)
            # pdb.set_trace()
            # target = target[0] - 1
            
            # loss = criterion(max, target)
            # loss = criterion(net_output, labels)
            # epoch_loss += loss.item()
            running_loss += loss.item()
            
            loss.backward()
            optimizer.step()

        epoch_loss = running_loss/len(train_inds)

        print('iteration ', e, 'loss ', epoch_loss)
        print('          prec rec  f1   acc')

        precision, recall, f_score, classification_accuracy = test_network(net, dataset, val_set)
        print('val set  ',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))

        precision, recall, f_score, classification_accuracy = test_network(net, dataset, train_set)
        print('train set',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))
        print()

    return net

In [0]:
def main():
    # create_clean_opseq_files() # should only run the first time this code is ran
    # print('Created clean opseq')
    # create_vuln_opseq_files() # should only run the first time this code is ran
    # print('Created vuln opseq')
    vuln, clean = read_dataset()

    dataset, train_set, val_set, test_set = split_dataset(vuln, clean)
    net = VulnerabilityDetectorNetwork()
    net.to(device)
    print(net)
    print('started training network')
    net = train_network(net, dataset, train_set, val_set)
    
    print('started testing network')
    precision, recall, f_score, classification_accuracy = test_network(net, dataset, test_set)
    print('test set',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))
    print()
    
# print('Running Test Case: Evaluation Function')
# test_case_result_eval_function = test_cases.test_evaluation_function(evaluate_network_performance)
# print('Test Case Result: ', test_case_result_eval_function)
# print()

# print('Running Test Case: Splitting Dataset')
# test_vuln,test_clean = read_dataset()
# test_case_result_split_dataset = test_cases.test_split_dataset(split_dataset, test_vuln, test_clean)
# print('Test Case Result: ', test_case_result_split_dataset)
# print()

# print('Running Test Case: Neural Network')
# test_case_result_network = test_cases.test_network(VulnerabilityDetectorNetwork)
# print('Test Case Result: ', test_case_result_network)
# print()

In [0]:
# if test_case_result_eval_function and test_case_result_split_dataset and test_case_result_network:
# if test_case_result_eval_function and test_case_result_split_dataset:
main()
# else:
#     print('one or more test cases failed - quitting')
#     quit()

VulnerabilityDetectorNetwork(
  (emb1): Embedding(205, 8)
  (conv1): Conv2d(1, 16, kernel_size=(1, 8), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop_out): Dropout(p=0.25)
  (lin1): Linear(in_features=8000, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=2, bias=True)
)
started training network
iteration  0 loss  0.254171869799283
          prec rec  f1   acc
val set   0.53 0.91 0.67 0.55
train set 0.95 0.91 0.93 0.93

iteration  1 loss  0.19091376609049962
          prec rec  f1   acc
val set   0.53 0.90 0.66 0.54
train set 0.95 0.91 0.93 0.93

iteration  2 loss  0.18673336116196354
          prec rec  f1   acc
val set   0.51 0.93 0.66 0.52
train set 0.94 0.93 0.94 0.94

iteration  3 loss  0.19399247897244967
          prec rec  f1   acc
val set   0.54 0.89 0.67 0.57
train set 

KeyboardInterrupt: ignored